
<h1> Information Regarding the project </h1>
<p> <h3>This project is designed to classify wildlife species </h3>
 Investigated by: - Mainuddin Alam Irteja
</p>

In [1]:
# Connecting to my google drive where the image set is kept
# Image set was used from kaggle
# Link to the image dataset:- https://www.kaggle.com/datasets/vishweshsalodkar/wild-animals/data
import os
from google.colab import drive
drive.mount('/content/gdrive')
rootDir = "/content/gdrive/MyDrive/Colab Notebooks/Wildlife Identifier Project/Animals"

Mounted at /content/gdrive


In [10]:
# Required modules for the project
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

In [11]:
# Loading and Augmenting the images
trainDir = os.path.join(rootDir, 'train')
validationDir= os.path.join(rootDir, 'validation')
testDir = os.path.join(rootDir, 'testing')

# Implementing data augmentation for the training set
# Augmentation is being implemented because orginal dataset is small
trainDatagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Apply rescaling for validation and test sets
validationDatagen = ImageDataGenerator(rescale=1./255)
testDatagen = ImageDataGenerator(rescale=1./255)

# Create the train, validation and test data generators
trainGenerator = trainDatagen.flow_from_directory(
    trainDir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validationGenerator = validationDatagen.flow_from_directory(
    validationDir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

testGenerator = testDatagen.flow_from_directory(
    testDir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 118 images belonging to 5 classes.
Found 32 images belonging to 5 classes.
Found 20 images belonging to 5 classes.


In [16]:
# Initialize and design model which will use CNN
wildlifeModel = Sequential()

# First block of Convolution layers and Maxpooling layer
wildlifeModel.add(Conv2D(32, (5, 5), activation='relu', input_shape=(224, 224, 3)))
wildlifeModel.add(Conv2D(32, (5, 5), activation='relu'))
wildlifeModel.add(BatchNormalization())
wildlifeModel.add(MaxPooling2D((3, 3)))

# Second block of Convolution layers and Maxpooling layer
wildlifeModel.add(Conv2D(64, (5, 5), activation='relu'))
wildlifeModel.add(Conv2D(64, (5, 5), activation='relu'))
wildlifeModel.add(BatchNormalization())
wildlifeModel.add(MaxPooling2D((3, 3)))

# Third block of Convolution layers and Maxpooling layer
wildlifeModel.add(Conv2D(128, (5, 5), activation='relu'))
wildlifeModel.add(Conv2D(128, (5, 5), activation='relu'))
wildlifeModel.add(BatchNormalization())
wildlifeModel.add(MaxPooling2D((3, 3)))

# Flattening the model to apply Dense layer
wildlifeModel.add(Flatten())

# Applying dense layer to fully connect each neurone
wildlifeModel.add(Dense(512, activation='relu'))
wildlifeModel.add(Dropout(0.5))

# Applying softmax to determine the 5 species
wildlifeModel.add(Dense(5, activation='softmax'))

# Compile the model
wildlifeModel.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy',  metrics=['accuracy'])

In [ ]:
# Training the model
wildlifeModel.fit(
    trainGenerator,
    steps_per_epoch=len(trainGenerator),
    epochs=60,
    validation_data=validationGenerator,
    validation_steps=len(validationGenerator)
)


Epoch 1/60
4/4 [==============================] - 53s 14s/step - loss: 4.1410 - accuracy: 0.2373 - val_loss: 1.6087 - val_accuracy: 0.1875
Epoch 2/60
4/4 [==============================] - 52s 13s/step - loss: 3.2844 - accuracy: 0.2119 - val_loss: 1.6121 - val_accuracy: 0.1875
Epoch 3/60
4/4 [==============================] - 49s 12s/step - loss: 2.5767 - accuracy: 0.3305 - val_loss: 1.6161 - val_accuracy: 0.1875
Epoch 4/60
4/4 [==============================] - 49s 13s/step - loss: 2.6097 - accuracy: 0.3051 - val_loss: 1.6227 - val_accuracy: 0.1875
Epoch 5/60
4/4 [==============================] - 52s 14s/step - loss: 2.1423 - accuracy: 0.3644 - val_loss: 1.6396 - val_accuracy: 0.1875
Epoch 6/60
4/4 [==============================] - 49s 12s/step - loss: 2.3708 - accuracy: 0.3305 - val_loss: 1.6739 - val_accuracy: 0.1875
Epoch 7/60
4/4 [==============================] - 50s 12s/step - loss: 2.1623 - accuracy: 0.3136 - val_loss: 1.7177 - val_accuracy: 0.1875
Epoch 8/60
4/4 [===========

In [15]:
# Evaluating the model
testLoss, testAccuracy = wildlifeModel.evaluate(testGenerator, steps=len(testGenerator))
print(f"Test accuracy: {testAccuracy * 100:.2f}%")

1/1 [==============================] - 2s 2s/step - loss: 3.3196 - accuracy: 0.2000
Test accuracy: 20.00%
